In [103]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
from operator import itemgetter
from itertools import compress
import matplotlib.pyplot as plt

import regex as re

In [140]:
with open('/Users/gabgilling/Dropbox/QMSS/Fall 2019/Practicum/df.pkl', 'rb') as f:
    df2= pickle.load(f)

In [141]:
df2.head()

,date_received,product,sub_product,issue,sub_issue,company,state,zip_code,consumer_consent_provided,submitted_via,date_sent_to_company,company_response,timely,consumer_disputed,complaint_id,tags,company_public_response,complaint_what_happened,processed,extracted
0,2019-10-10T00:00:00.000,"Credit reporting, credit repair services, or o...",Other personal consumer report,Incorrect information on your report,Public record information inaccurate,STERLING INFOSYSTEMS INC,CA,913XX,Consent provided,Web,2019-10-10T00:00:00.000,Closed with explanation,Yes,N/A,3401282,NaN,Company has responded to the consumer and the ...,I just recieved my background check tonight / ...,"[recieved, background, check, simply, applying...",I feel that possibly someone is using my soci...
1,2019-10-10T00:00:00.000,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,221XX,Consent provided,Web,2019-10-10T00:00:00.000,Closed with explanation,Yes,N/A,3401413,NaN,Company has responded to the consumer and the ...,I recently had 4 30-day late payments on my cr...,"[recently, late, payment, credit, report, emai...",I recently had 4 30-day late payments on my cr...
2,2019-10-10T00:00:00.000,"Payday loan, title loan, or personal loan",Installment loan,Charged fees or interest you didn't expect,NaN,"Orions Management Group, LLC",UT,844XX,Consent provided,Web,2019-10-10T00:00:00.000,Closed with explanation,Yes,N/A,3401037,NaN,Company disputes the facts presented in the co...,This account was opened in of 2015 fraudulent...,"[account, fraudulently, identity, stolen, seve...",I asked them to provide me with paperwork tha...
3,2019-10-09T00:00:00.000,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,"Credit Protection Association, L.P.",SC,298XX,Consent provided,Web,2019-10-09T00:00:00.000,Closed with explanation,Yes,N/A,3399879,Servicemember,Company has responded to the consumer and the ...,I am a victim of identity theft and this debt ...,"[victim, identity, theft, debt, belong, please...",I am a victim of identity theft and this debt ...
4,2019-10-09T00:00:00.000,Debt collection,Medical debt,Written notification about debt,Didn't receive enough information to verify debt,AmeriCollect,FL,334XX,Consent provided,Web,2019-10-09T00:00:00.000,Closed with explanation,Yes,N/A,3400783,NaN,Company believes it acted appropriately as aut...,When the original debtor had this account I...,"[original, debtor, account, even, touch, anyon...",This collection agency is not applying to the...


In [134]:
df['extracted'] = ''

## Word Embeddings approach

inspired from: http://www.lumenai.fr/blog/quick-review-on-text-clustering-and-text-similarity-approaches

In [81]:
from gensim.models import word2vec

def w2v(s1,s2,wordmodel = wordmodel):
    
    s1words=s1.split()
    s2words=s2.split()
    s1wordsset=set(s1words)
    s2wordsset=set(s2words)
    vocab = wordmodel.vocab # the vocabulary considered in the word embeddings
    
    s1words = [word for word in s1words if word in vocab]
    s2words = [word for word in s2words if word in vocab]
    
    if len(s1words) == 0: # sometimes we have an empty narrative, mitigate risk and assign 0 score
        return 0

    return wordmodel.n_similarity(s1words, s2words)


In [6]:
df['complaint_what_happened'].iloc[0]

"I just recieved my background check tonight / I simply was applying for a job at   and just recieved that I'm a   which is absolutely wrong. Which I wan na dispute and get rid of from my record. I feel that possibly someone is using my social or other information."

In [15]:
s1 = df['complaint_what_happened'].iloc[0].split('.')[0]

In [16]:
s2 = df['issue'].iloc[0]

In [14]:
wordmodelfile='https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
from gensim import models
wordmodel=models.KeyedVectors.load_word2vec_format(wordmodelfile, binary=True)

In [45]:
w2v(s1,s2,wordmodel)

0.40281254

In [73]:
scores = []
for sent in df['complaint_what_happened'].iloc[0].split('.'):
    if sent != '':
        print(w2v(sent, s2))
        scores.append(w2v(sent, s2))

scores.index(max(scores))
df['complaint_what_happened'].iloc[0].split('.')[scores.index(max(scores))]

0.40281254
0.34592712
0.5486311


' I feel that possibly someone is using my social or other information'

In [135]:
split = df['complaint_what_happened'].apply(lambda x: [sent for sent in x.split('.') if sent != ''])

In [136]:
for i in range(len(df)):
    if i % 1000 == 0: # I use print statements to keep track of the progress of the algorithm
        print(i)
    scores = []
    s2 = df['issue'].iloc[i]
    for sent in split.iloc[i]:
        scores.append(w2v(sent, s2))
    try:
        df['extracted'].iloc[i] = split.iloc[i][scores.index(max(scores))]
    except:
        print('Gotcha at loc: ' + str(i))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
Gotcha at loc: 72573
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000


In [137]:
df.head()

,date_received,product,sub_product,issue,sub_issue,company,state,zip_code,consumer_consent_provided,submitted_via,date_sent_to_company,company_response,timely,consumer_disputed,complaint_id,tags,company_public_response,complaint_what_happened,processed,extracted
0,2019-10-10T00:00:00.000,"Credit reporting, credit repair services, or o...",Other personal consumer report,Incorrect information on your report,Public record information inaccurate,STERLING INFOSYSTEMS INC,CA,913XX,Consent provided,Web,2019-10-10T00:00:00.000,Closed with explanation,Yes,N/A,3401282,NaN,Company has responded to the consumer and the ...,I just recieved my background check tonight / ...,"[recieved, background, check, simply, applying...",I feel that possibly someone is using my soci...
1,2019-10-10T00:00:00.000,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,221XX,Consent provided,Web,2019-10-10T00:00:00.000,Closed with explanation,Yes,N/A,3401413,NaN,Company has responded to the consumer and the ...,I recently had 4 30-day late payments on my cr...,"[recently, late, payment, credit, report, emai...",I recently had 4 30-day late payments on my cr...
2,2019-10-10T00:00:00.000,"Payday loan, title loan, or personal loan",Installment loan,Charged fees or interest you didn't expect,NaN,"Orions Management Group, LLC",UT,844XX,Consent provided,Web,2019-10-10T00:00:00.000,Closed with explanation,Yes,N/A,3401037,NaN,Company disputes the facts presented in the co...,This account was opened in of 2015 fraudulent...,"[account, fraudulently, identity, stolen, seve...",I asked them to provide me with paperwork tha...
3,2019-10-09T00:00:00.000,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,"Credit Protection Association, L.P.",SC,298XX,Consent provided,Web,2019-10-09T00:00:00.000,Closed with explanation,Yes,N/A,3399879,Servicemember,Company has responded to the consumer and the ...,I am a victim of identity theft and this debt ...,"[victim, identity, theft, debt, belong, please...",I am a victim of identity theft and this debt ...
4,2019-10-09T00:00:00.000,Debt collection,Medical debt,Written notification about debt,Didn't receive enough information to verify debt,AmeriCollect,FL,334XX,Consent provided,Web,2019-10-09T00:00:00.000,Closed with explanation,Yes,N/A,3400783,NaN,Company believes it acted appropriately as aut...,When the original debtor had this account I...,"[original, debtor, account, even, touch, anyon...",This collection agency is not applying to the...


In [96]:
print(df['issue'].iloc[3])
print(df['extracted'].iloc[3])

Attempts to collect debt not owed
I am a victim of identity theft and this debt does not belong to me


In [98]:
print(df['issue'].iloc[83730])
print(df['complaint_what_happened'].iloc[83730])


Struggling to pay your loan
Victimized by predatory lending when purchased used car. My credit score was not even that low to merit a sub prime lender. The lender immediately started imputing fees and I got further and further behind. They called 10-20 times a day, seven days a week and when I asked them to stop but rather put it in writing they said I had a cease and desist order, of which I never did. They said I missed two payments ( $ /mo ) in the life of the loan and on / they repossessed my car. I purchased the car / for {$12000.00} and to date have paid more than {$10000.00}. According to the lender, because of the " simple interest loan '' and my outrageous APR ( 25 % ) I still owe them {$11000.00} and now my credit is ruined and I have no car. HELP PLEASE!
 According to the lender, because of the " simple interest loan '' and my outrageous APR ( 25 % ) I still owe them {$11000


As we can see, periods are included in the dollar amounts, causing the string to be split in the wrong locations. 

I use regex to substitute out the periods and the extra 00s with them too.

Then I rerun the code with the updated narratives.

In [130]:
re.sub('\.00', '', df['complaint_what_happened'].iloc[83730])

'Victimized by predatory lending when purchased used car. My credit score was not even that low to merit a sub prime lender. The lender immediately started imputing fees and I got further and further behind. They called 10-20 times a day, seven days a week and when I asked them to stop but rather put it in writing they said I had a cease and desist order, of which I never did. They said I missed two payments ( $ /mo ) in the life of the loan and on / they repossessed my car. I purchased the car / for {$12000} and to date have paid more than {$10000}. According to the lender, because of the " simple interest loan \'\' and my outrageous APR ( 25 % ) I still owe them {$11000} and now my credit is ruined and I have no car. HELP PLEASE!'

In [132]:
# apply that to all rows in the narratives
df['complaint_what_happened'] = df['complaint_what_happened'].apply(lambda x : re.sub('\.00', '', x))

In [138]:
print(df['complaint_what_happened'].iloc[83730])

Victimized by predatory lending when purchased used car. My credit score was not even that low to merit a sub prime lender. The lender immediately started imputing fees and I got further and further behind. They called 10-20 times a day, seven days a week and when I asked them to stop but rather put it in writing they said I had a cease and desist order, of which I never did. They said I missed two payments ( $ /mo ) in the life of the loan and on / they repossessed my car. I purchased the car / for {$12000} and to date have paid more than {$10000}. According to the lender, because of the " simple interest loan '' and my outrageous APR ( 25 % ) I still owe them {$11000} and now my credit is ruined and I have no car. HELP PLEASE!


In [139]:
#df.to_pickle('/Users/gabgilling/Dropbox/QMSS/Fall 2019/Practicum/df.pkl')